<a href="https://colab.research.google.com/github/IshaqKHATTAK/Natural-Language-Processing/blob/main/NLP_paper_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import numpy as np
import pandas as pd

In [11]:
#!pip install gensim


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
#!unzip '/content/drive/MyDrive/coviddata.zip'

Archive:  /content/drive/MyDrive/coviddata.zip
  inflating: Covid-19PublicTweets.xlsx  
  inflating: TweetsCovid.csv         
  inflating: TweetsCovid-19Datasetupdated.csv  


###reading data

In [15]:
path = '/content/TweetsCovid-19Datasetupdated.csv'
column = ['Tweet','Date']
data = pd.read_csv(path,names=column, encoding='cp1252')

In [16]:
data.head(2)

,Tweet,Date
0,'Not a Secret Cult \n\n#Scientology #Vaccines...,9/26/2021
1,"""Another one of Dr John Campbell's videos whe...",9/26/2021


In [ ]:
len(data)

156539

###drop out retweeted 

In [17]:
#df = data[data.duplicated(['Tweet'], keep=False)]
data = data.drop_duplicates()
len(data)

155490

In [18]:
!pip install emoji
import re
import emoji
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import PorterStemmer, WordNetLemmatizer
nltk.download('wordnet')


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234911 sha256=74ba2fd1b32a62497a95f2e4e4041b1be912cf8cb9cba814fb18d01b955beede
  Stored in directory: /root/.cache/pip/wheels/02/3d/88/51a592b9ad17e7899126563698b4e3961983ebe85747228ba6
Successfully built emoji


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [19]:
sw = stopwords.words('english')
ps = PorterStemmer()
wl = WordNetLemmatizer()

def to_lower(text):
  text = text.lower()
  return text

def remove_URL(text):
  text = re.sub(r'https?://\S+','',text)
  return text

def remove_punctuation(text):
  text = re.sub(r'[^\w\s]','',text)
  return text

def remove_emji(text):
  text = emoji.demojize(text, delimiters=('', ''))
  return text

def tokenize_stopword_lematize_stemed(text):

  text = word_tokenize(text) 
  word_list = [w for w in text if w not in sw]
  lematized = [wl.lemmatize(w) for w in word_list]
  stemmed = [ps.stem(w) for w in lematized]
  return ' '.join(stemmed)



In [20]:
data['Tweet'] = data['Tweet'].apply(to_lower)
data['Tweet'] = data['Tweet'].apply(remove_URL)
data['Tweet'] = data['Tweet'].apply(remove_punctuation)
data['Tweet'] = data['Tweet'].apply(remove_emji)
data['Tweet'] = data['Tweet'].apply(tokenize_stopword_lematize_stemed)
data

,Tweet,Date
0,secret cult nnscientolog vaccin vaccin covid19...,9/26/2021
1,anoth one dr john campbel video spit tack say ...,9/26/2021
2,slamdunk nobrain argument 13 america refus vac...,9/26/2021
3,natur inmun rule vaccin,9/26/2021
4,long list antivaxx deathb see error way plead ...,9/26/2021
...,...,...
156534,unhealthytruth peopl better wake f npfizer mod...,3/27/2022
156535,20220326480 839 751 nvaccin tool way spread co...,3/27/2022
156536,far stupidest respons ever gotten question kno...,3/27/2022
156537,industrialist philanthropist john lumic releas...,3/27/2022


In [27]:
from textblob import TextBlob

sentiment_textblob = [] #positive --> 1 negtive-->2 netral --> 0

def sentiment_TexBlob(text):
  # Create a TextBlob object
  blob = TextBlob(text)
  # Perform sentiment analysis
  sentiment = blob.sentiment.polarity
  if sentiment == 0.0:
    sentiment_textblob.append(0)
  elif sentiment < 0:
    sentiment_textblob.append(2)
  else:
    sentiment_textblob.append(1)
  

data['Tweet'].apply(sentiment_TexBlob)

0         None
1         None
2         None
3         None
4         None
          ... 
156534    None
156535    None
156536    None
156537    None
156538    None
Name: Tweet, Length: 155490, dtype: object

In [31]:
data['sentiment textblob'] = sentiment_textblob
data.head(3)

,Tweet,Date,sentiment textblob
0,secret cult nnscientolog vaccin vaccin covid19...,9/26/2021,2
1,anoth one dr john campbel video spit tack say ...,9/26/2021,0
2,slamdunk nobrain argument 13 america refus vac...,9/26/2021,0


In [ ]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

In [4]:
sentiment_vader = []
def sentiment_vader(text):
  van = SentimentIntensityAnalyzer()
  sntment = van.polarity_scores(text)
  return max(sntment, key=sntment.get)
data['sentiment vader'] = data['Tweet'].apply(sentiment_vader)
data.head(3)

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


{'neg': 0.0, 'neu': 0.323, 'pos': 0.677, 'compound': 0.6369}

In [35]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data['Tweet'], data['sentiment textblob'])

In [36]:
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer


tfidf = TfidfVectorizer()
tfidf.fit(data['Tweet'])

tfidf_vec_train = tfidf.transform(x_train)
tfidf_vec_test = tfidf.transform(x_test)

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
vec = cv.fit(data['Tweet'])

cv_vec_train = vec.transform(x_train)
cv_vec_test = vec.transform(x_test)

array([[0, 1, 1, 1, 0, 0, 1, 0, 1],
       [0, 2, 0, 1, 0, 1, 1, 0, 1],
       [1, 0, 0, 1, 1, 0, 1, 1, 1],
       [0, 1, 1, 1, 0, 0, 1, 0, 1]])

In [ ]:
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# Convert the corpus to TaggedDocuments
tagged_corpus = [TaggedDocument(doc.split(), [i]) for i, doc in enumerate(data['Tweet'])]

# Train the Doc2Vec model
model = Doc2Vec(tagged_corpus, vector_size=50, window=2, min_count=1, workers=4, epochs=100)

# Get the vector representation of the first document
d2v_vec_train = model.infer_vector(x_train.words)
d2v_vec_test = model.infer_vector(x_test.words)


In [ ]:
#TextBlob + TF-IDF + LinearSVC  tfidf_vec
from sklearn.svm import SVC

SVM = SVC()
SVM.fit(tfidf_vec_train, y_train)
y_pred = SVM.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score,f1_score

accuracy = accuracy_score(y_pred, x_test)
f1 = f1_score(y_pred, x_test)
precision = precision_score(y_pred, x_test)

print(f'accuracy: {accuracy} precision: {precision} f1: {f1}')

In [ ]:
svm2 = SVC()

svm2.fit(cv_vec_train, y_train)
y_pred = svm2.predict(x_test)

In [ ]:
collect_word = ['covid', 'covid19', 'vaccine', 'corona', 'virus']
def remove_collection(text):
  for word in collect_word:
    text = re.sub(word,'',text)
  return text

data['Tweet'] = data['Tweet'].apply(remove_collection)

In [ ]:
from collections import Counter
dictionary = []
def to_tokens(text):
  text = word_tokenize(text)
  for word in text:
    dictionary.append(word)
  return text
to_tokens('this is ishaq you have you khan from call.from this as you')

bag_of_word = Counter(dictionary)

In [ ]:
bag_of_word.most_common(2)

[('you', 3), ('this', 2)]